<a href="https://colab.research.google.com/github/stefanoiervese/DL_Project/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
path='/content/drive/MyDrive/mathematics_dataset-v1.0/train-easy/arithmetic__add_or_sub.txt'
#os.listdir(path)
with open(path, "r") as file:
    # Leggi il contenuto del file
    content = file.read()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import random
import re

data_list = [x for x in content.split('\n')]
data_list=data_list[:-1]
len_data=len(data_list)
quest=[]
ans=[]
for i in range(len_data):
  if(i%2==0):
    quest.append(data_list[i])
  else:
    ans.append(data_list[i])
coppie = list(zip(quest,ans))
random.shuffle(coppie)
quest, ans=zip(*coppie)
l=int(len(quest)/3)
train_q=quest[:2*l]
test_q=quest[2*l:]
train_a=ans[:2*l]
test_a=ans[2*l:]

voc_size=len([sentence.split() for sentence in data_list])


def crea_vocabolario(frasi):
    vocabolario = set()

    for frase in frasi:
        # Utilizza un'espressione regolare per estrarre parole, numeri e simboli dalla frase
        parole = re.findall(r'\d|\.|[-=+*/()?]|\S+|\?', frase.lower())

        # Aggiungi le parole al vocabolario
        vocabolario.update(parole)


    return list(vocabolario)+['unknown']

v=crea_vocabolario(data_list)




# Token

In [3]:
class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.word_to_id = {word: idx for idx, word in enumerate(vocab)}

    def tokenize(self, text):
        tokens = re.findall(r'\d|\.|[-=+*/()?]|\S+', text)



        token_id = []
        unknown_tokens = []

        new_tokens = []
        for token in tokens:
          flag=0
          if '.' in token and token != '.':
            token=token=token.split('.')[0]
            flag=1
          if token in self.word_to_id:
            token_id.append(self.word_to_id[token])
          else:
                token_id.append(self.word_to_id['unknown'])
                unknown_tokens.append(token)
          if(flag==1):
            token_id.append(self.word_to_id['.'])


        if unknown_tokens:
            print("Parole sconosciute:", unknown_tokens)

        return token_id



tokenizer = Tokenizer(v)

text = "What is 3?"
token_ids = tokenizer.tokenize(text)
print("Token IDs:", token_ids)

Parole sconosciute: ['What']
Token IDs: [38, 37, 9, 4]


# DataLoader

In [4]:
import torch
qt=[]
at=[]
for x in train_q:
  qt.append(torch.tensor(tokenizer.tokenize(x.lower())))
for x in train_a:
  at.append(torch.tensor(tokenizer.tokenize(x.lower())))

qtest=[]
atest=[]
for x in test_q:
  qtest.append(torch.tensor(tokenizer.tokenize(x.lower())))
for x in test_a:
  atest.append(torch.tensor(tokenizer.tokenize(x.lower())))






In [ ]:
pip install pytorch_lightning

In [18]:

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence, pad_sequence


class Dataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.answers[idx]




voc_size=100
src_data = torch.randint(1, voc_size, (3264, 6))
tg_data=torch.randint(1,voc_size,(3264,8))
fake_dataset=Dataset(src_data,tg_data)
fake_loader=DataLoader(fake_dataset,batch_size=32,shuffle=True)

max_length1 = max(len(tensor) for tensor in qt)
max_length2 = max(len(tensor) for tensor in at)
max_length=max(max_length1,max_length2)
qtp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in qt], batch_first=True)
atp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in at], batch_first=True)


max_length1 = max(len(tensor) for tensor in qtest)
max_length2 = max(len(tensor) for tensor in atest)
max_length=max(max_length1,max_length2)
qtestp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in qtest], batch_first=True)
atestp = pad_sequence([torch.cat([tensor, torch.zeros(max_length - len(tensor),dtype=torch.int)]) for tensor in atest], batch_first=True)


batch_size = 3

train_dataset = Dataset(qtp,atp)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
'''
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: (
    pad_sequence([item[0] for item in batch], batch_first=True),
    pad_sequence([item[1] for item in batch], batch_first=True)
))'''
test_dataset = Dataset(qtestp,atestp)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)


# Transformer

In [25]:

import torch
import torch.nn as nn
import math
import pytorch_lightning as pl
import torch.optim as optim

class MultiHeadAttention(pl.LightningModule):
    def __init__(self, emb_dim, num_heads,dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert emb_dim % num_heads == 0, "emb_dim must be divisible by num_heads"

        self.emb_dim = emb_dim
        self.num_heads = num_heads
        self.head_dim = self.emb_dim // num_heads

        # Inizializzazione dei moduli lineari per proiettare Q, K, V e l'output.
        self.W_q = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_k = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_v = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_o = nn.Linear(self.emb_dim, self.emb_dim)


    def forward(self, q,k,v):
        q = self.split_heads(self.W_q(q))  #64,2,22,2
        k = self.split_heads(self.W_k(k))  #64,2,2,22
        v = self.split_heads(self.W_v(v))


        att = self.att_score(q, k, v)  #62,2,22,22
        out = self.W_o(self.combine_heads(att))

        return out

    def att_score(self, q, k, v):
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        attn_scores=self.apply_mask(attn_scores)




        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, v)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, head_dim = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.emb_dim)

    def apply_mask(self,tensor):
      for i in range(tensor.size(0)):
        for j in range(tensor.size(1)):
          matrix = tensor[i, j]

          mask = torch.triu(torch.ones(matrix.size(0), matrix.size(1)), diagonal=0,).t().to('cuda:0')

          matrix = matrix * mask+ (-1e9) * (1 - mask)
          tensor[i, j] = matrix
          return tensor







class EncoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(EncoderLayer, self).__init__()

        self.multihead_attention = MultiHeadAttention(emb_dim, num_heads)

        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        att = self.multihead_attention(x,x,x)

        add_nor = self.layer_norm1(x + self.dropout(att))
        ff_out = self.feedforward(add_nor)
        out = self.layer_norm2(add_nor + self.dropout(ff_out))


        return out





class Encoder(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Encoder, self).__init__()

        self.layers = nn.ModuleList([
            EncoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class DecoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # Self-Attention Layer (Auto-Attention)
        self.self_attention = MultiHeadAttention(emb_dim, num_heads)

        # Cross-Attention Layer (Attenzione incrociata con l'encoder)
        self.cross_attention = MultiHeadAttention(emb_dim, num_heads)

        # Feedforward Neural Network Layer
        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        # Layer Normalization
        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.layer_norm3 = nn.LayerNorm(emb_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output):

        attn_output = self.self_attention(x, x, x)
        out1 = self.layer_norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attention(out1, encoder_output, encoder_output)
        out2 = self.layer_norm2(out1 + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        out3 = self.layer_norm3(out2 + self.dropout(ff_output))
        return out3

class Decoder(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Decoder, self).__init__()


        self.layers = nn.ModuleList([
            DecoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, encoder_output):
        for layer in self.layers:
            x = layer(x, encoder_output)
        return x

import pytorch_lightning as pl
import torch
import torch.nn as nn

class Transformer(pl.LightningModule):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=4, num_heads=2, num_layers=6, d_ff=256, max_seq_length=22, dropout=0.1):
        super(Transformer, self).__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = self.positional_encoding(max_seq_length=max_length, d_model=d_model)

        self.transformer_encoder = Encoder(self.d_model,self.num_heads

        )
        self.transformer_decoder =Decoder(self.d_model,self.num_heads
        )

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        #self.positional_encoding = self.positional_encoding(max_seq_length, d_model)  # Correggi l'argomento qui



    def forward(self, src, tgt):
        src_embedded = self.dropout(self.src_embedding(src))

        tgt_embedded = self.dropout(self.tgt_embedding(tgt))


        enc_output = self.transformer_encoder(src_embedded)
        dec_output = self.transformer_decoder(tgt_embedded, enc_output)

        output = self.fc(dec_output)
        return output


    def training_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt)
        tgt_mask = (tgt != 0)
        loss = nn.CrossEntropyLoss(ignore_index=0)(output.view(-1, output.size(-1)), tgt.view(-1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt)
        tgt_mask = (tgt != 0)
        loss = nn.CrossEntropyLoss(ignore_index=0)(output.view(-1, output.size(-1)), tgt.view(-1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

    def test_step(self, batch, batch_idx):
      src, tgt = batch
      output = self(src, tgt)
      tgt_mask = (tgt != 0)
      loss = nn.CrossEntropyLoss(ignore_index=0)(output.view(-1, output.size(-1)), tgt.view(-1))
      self.log('test_loss', loss)  # Registra la loss del test
      return loss




    def positional_encoding(self, max_seq_length, d_model):

        positional_encoding = torch.zeros(max_seq_length, d_model)
        pos = torch.arange(0, max_seq_length, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        positional_encoding[:, 0::2] = torch.sin(pos * div_term)
        positional_encoding[:, 1::2] = torch.cos(pos * div_term)
        return positional_encoding






voc_len=len(v)
t=Transformer(voc_len,voc_len)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t.to(device)
print(device)
# Addestra il modello

trainer = pl.Trainer(max_epochs=5)  # Modifica il numero di epoche come desiderato
trainer.fit(t, train_loader)
trainer.test(test_loader)

#model_path='/content/drive/MyDrive/mathematics_dataset-v1.0/model'


#torch.save(t.state_dict(), model_path)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type      | Params
--------------------------------------------------
0 | src_embedding       | Embedding | 156   
1 | tgt_embedding       | Embedding | 156   
2 | transformer_encoder | Encoder   | 2.3 K 
3 | transformer_decoder | Decoder   | 2.9 K 
4 | fc                  | Linear    | 195   
5 | dropout             | Dropout   | 0     
--------------------------------------------------
5.7 K     Trainable params
0         Non-trainable params
5.7 K     Total params
0.023     Total estimated model params size (MB

cuda


Training: 0it [00:00, ?it/s]

TypeError: ignored

In [ ]:
import torch

# Crea un tensore di esempio con matrici 3x3 duplicate
tensor = torch.tensor([[[[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 9]],
                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]],
                        [[19, 20, 21],
                         [22, 23, 24],
                         [25, 26, 27]],
                        [[28, 29, 30],
                         [31, 32, 33],
                         [34, 35, 36]],
                        [[37, 38, 39],
                         [40, 41, 42],
                         [43, 44, 45]]],
                       [[[46, 47, 48],
                         [49, 50, 51],
                         [52, 53, 54]],
                        [[55, 56, 57],
                         [58, 59, 60],
                         [61, 62, 63]],
                        [[64, 65, 66],
                         [67, 68, 69],
                         [70, 71, 72]],
                        [[73, 74, 75],
                         [76, 77, 78],
                         [79, 80, 81]],
                        [[82, 83, 84],
                         [85, 86, 87],
                         [88, 89, 90]]]])

# Imposta tutti i valori sopra la diagonale principale di ciascuna matrice a zero
for i in range(tensor.size(0)):
    for j in range(tensor.size(1)):
        matrix = tensor[i, j]
        mask = torch.triu(torch.ones(matrix.size(0), matrix.size(1)), diagonal=0).t()
        matrix = matrix * mask+ (-1e9) * (1 - mask)
        tensor[i, j] = matrix

print("Tensore originale:")
print(tensor)



In [ ]:
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Crea una maschera booleana per nascondere alcuni elementi nel tensore
mask = torch.tensor([[0, 1, 0],  # Nascondi la seconda colonna
                     [1, 0, 1],  # Nascondi la prima e la terza colonna
                     [0, 1, 0]]) # Nascondi la seconda colonna

# Applica la maschera al tensore utilizzando masked_fill
masked_tensor = tensor.masked_fill(mask == 0, -1)

print("Tensore originale:")
print(tensor)

print("\nMaschera:")
print(mask)

print("\nTensore mascherato:")
print(masked_tensor)

In [ ]:
def generate_square_subsequent_mask(size):
    mask = torch.triu(torch.ones(size, size)).t()
    return mask
generate_square_subsequent_mask(3)

In [ ]:
model_path='/content/drive/MyDrive/model'
t.load_state_dict(torch.load(model_path))
t.eval()

In [ ]:
torch.save(t.state_dict(), 'modello_dopo_training.pth')

In [ ]:
trainer.test(t,test_loader)



# TP-Transformer

In [ ]:
class TPMultiHeadAttention(pl.LightningModule):
    def __init__(self, emb_dim, num_heads,dropout=0.1):
        super(TPMultiHeadAttention, self).__init__()
        assert emb_dim % num_heads == 0, "emb_dim must be divisible by num_heads"

        self.emb_dim = emb_dim
        self.num_heads = num_heads
        self.head_dim = self.emb_dim // num_heads

        # Inizializzazione dei moduli lineari per proiettare Q, K, V e l'output.
        self.W_q = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_k = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_v = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_r = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_o = nn.Linear(self.emb_dim, self.emb_dim)


    def forward(self, q,k,v,r):
        q = self.split_heads(self.W_q(q))
        k = self.split_heads(self.W_k(k))
        v = self.split_heads(self.W_v(v))
        r = self.split_heads(self.W_r(r))

        att = self.att_score(q, k, v, mask)
        filler = torch.tensordot(att, r, dims=0)
        diagonal = torch.diagonal(filler)


        out = self.W_o(self.combine_heads(att))

        return out

    def att_score(self, q, k, v):
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, v)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, head_dim = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.emb_dim)






class TPEncoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(TPEncoderLayer, self).__init__()

        self.multihead_attention = TPMultiHeadAttention(emb_dim, num_heads)

        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        att = self.multihead_attention(x,x,x,x, mask)

        add_nor = self.layer_norm1(x + self.dropout(att))
        ff_out = self.feedforward(add_nor)
        out = self.layer_norm2(add_nor + self.dropout(ff_out))


        return out


class TPDecoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(TPDecoderLayer, self).__init__()

        # Self-Attention Layer (Auto-Attention)
        self.self_attention = TPMultiHeadAttention(emb_dim, num_heads)

        # Cross-Attention Layer (Attenzione incrociata con l'encoder)
        self.cross_attention = TPMultiHeadAttention(emb_dim, num_heads)

        # Feedforward Neural Network Layer
        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        # Layer Normalization
        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.layer_norm3 = nn.LayerNorm(emb_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):

        attn_output = self.self_attention(x, x, x,x, tgt_mask)
        out1 = self.layer_norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attention(out1, encoder_output, encoder_output, encoder_output, src_mask)
        out2 = self.layer_norm2(out1 + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        out3 = self.layer_norm3(out2 + self.dropout(ff_output))
        return out3


voc_len=39
t2=Transformer(voc_len,voc_len)
trainer2 = pl.Trainer(max_epochs=1)  # Modifica il numero di epoche come desiderato
trainer2.fit(t2, train_loader)